# Load Model

## At BFRU Dataset

In [ ]:
!wget https://raw.githubusercontent.com/adailtonaraujo/app_review_analysis/master/Classification/Models/MNB_RoBERTa_BFRU.pkl

In [ ]:
pkl_filename = "MNB_RoBERTa_BFRU.pkl"

## At ARE Dataset


In [ ]:
!wget https://raw.githubusercontent.com/adailtonaraujo/app_review_analysis/master/Classification/Models/SVM_RoBERTa_ARE.pkl

In [ ]:
pkl_filename = "SVM_RoBERTa_ARE.pkl"

## LOAD


In [ ]:
import pickle
with open(pkl_filename, 'rb') as file:
  clf = pickle.load(file)

# Represent User Text

In [ ]:
!pip install -U sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

In [ ]:
import pandas as pd
def WordEmbeddings(texts, algorithm):

  model = SentenceTransformer('roberta-large-nli-stsb-mean-tokens')

  if type(texts) == pd.core.series.Series:
    sentences = texts.replace(['\t','\n','\r'], [' ',' ',' '], regex=True)
  else:
    sentences = texts

  sentence_embeddings = model.encode(list(sentences))

  if algorithm == 'MNB':
    return np.abs(np.min(sentence_embeddings)) + sentence_embeddings
  elif algorithm == 'SVM':
    return sentence_embeddings

The variable *txts* can be a dataframe column or a list of strings

# DEMO

In [ ]:
txts = ['the app always crashes !!!!!!!!!!', 'I loved this app!!']

data = WordEmbeddings(txts,pkl_filename.split('_')[0])

predicted_classes = clf.predict(data)
predicted_classes

array(['pbr', 'irr'], dtype='<U3')

# Classification Metrics


**y_test_class on BFRU:**


*   Bug
*   Feature
*   User Experience
*   Rating

**y_test_class on ARE:**

*   pbr
*   iqr
*   irr






In [ ]:
y_test_class = []

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test_class, predicted_classes, output_dict=False))